# Linear Elasticity with Adjoint Gradients

This example demonstrates the capabilities of the framework in modern engineering simulations by investigating the deformation of elastic bodies. The governing equations for these problems are given by the equations of linear elasticity.

We define the problem on a 3D domain $\Omega$ representing a beam with length $L$ and width $W$. The beam is clamped on the left side of the domain and a body force acts on the beam.

Small elastic deformations of the body $\Omega$ under isotropic elastic conditions are described by the displacement vector field $\mathbf{u}(\mathbf{x}) : \Omega \to \mathbb{R}^3$ with the equations:

$$
\begin{align}
-\nabla \cdot \boldsymbol{\sigma}(\mathbf{u}) &= \mathbf{f} \quad \text{in } \Omega \\
\boldsymbol{\sigma}(\mathbf{u}) &= \lambda \text{tr}(\boldsymbol{\varepsilon}(\mathbf{u}))\mathbf{I} + 2\mu\boldsymbol{\varepsilon}(\mathbf{u}) \\
\boldsymbol{\varepsilon}(\mathbf{u}) &= \frac{1}{2}(\nabla\mathbf{u} + \nabla\mathbf{u}^\mathrm{T})
\end{align}
$$

where $\boldsymbol{\sigma}(\mathbf{u})$ is the stress tensor, $\boldsymbol{\varepsilon}(\mathbf{u})$ is the strain tensor, $\lambda$ and $\mu$ are the Lamé parameters, $\mathbf{f}$ is the body force per unit volume, and $\mathbf{I}$ is the identity tensor.

We define the weak form of the linear elasticity problem on a vector-valued ansatz space $V$.

Find $\mathbf{u} \in V$ such that:

$$
a(\mathbf{u}, \mathbf{v}) = L(\mathbf{v}) \quad \forall \mathbf{v} \in V
$$

where:

$$
\begin{align}
a(\mathbf{u}, \mathbf{v}) &= \int_\Omega \boldsymbol{\sigma}(\mathbf{u}) : \boldsymbol{\varepsilon}(\mathbf{v}) \, d\mathbf{x} \\
L(\mathbf{v}) &= \int_\Omega \mathbf{f} \cdot \mathbf{v} \, d\mathbf{x} + \int_\Gamma \mathbf{T} \cdot \mathbf{v} \, d\mathbf{s}
\end{align}
$$

In this form, the colon denotes the Frobenius inner product and $\mathbf{T}$ is the traction vector at the boundary.

We solve this problem using the residual equation:

$$
F(\mathbf{u}) = a(\mathbf{u}, \mathbf{v}) - L(\mathbf{v}) = 0
$$

In [ ]:
import numpy as np
import ufl
from basix.ufl import element
from dolfinx import fem, io, mesh, nls
from mpi4py import MPI
from petsc4py.PETSc import ScalarType

from dolfinx_adjoint import *

We define the geometry and material parameters for a cantilever beam:

In [ ]:
# Create computational graph
graph_ = Graph()

# Scaled variable
L = 1  # Length of the beam
W = 0.1  # Width of the beam
rho = 1  # Density
delta = W / L
gamma = 0.4 * delta**2
g = gamma

In [ ]:
domain = mesh.create_box(
    MPI.COMM_WORLD,
    [np.array([0, 0, 0]), np.array([L, W, W])],
    [30, 10, 10],
    cell_type=mesh.CellType.hexahedron,
)

vector_element = element("Lagrange", domain.basix_cell(), 1, shape=(3,))
V = fem.functionspace(domain, vector_element)
ds = ufl.Measure("ds", domain=domain)

In [ ]:
u = fem.Function(V, name="Deformation", graph=graph_)
v = ufl.TestFunction(V)

# Loads and boundary tractions
f = fem.Constant(domain, ScalarType((0, 0, -rho * g)))
T = fem.Constant(domain, ScalarType((0, 0, 0)))

# Lamé parameters (attached to the graph for gradient computation)
lambda_ = fem.Constant(domain, ScalarType(1.0), graph=graph_)
mu = fem.Constant(domain, ScalarType(1.25), graph=graph_)

In [ ]:
a = (
    ufl.inner(
        lambda_ * ufl.nabla_div(u) * ufl.Identity(len(u))
        + 2 * mu * ufl.sym(ufl.grad(u)),
        ufl.sym(ufl.grad(v)),
    )
    * ufl.dx
)
L = ufl.dot(f, v) * ufl.dx + ufl.dot(T, v) * ds
F = a - L

In [ ]:
# Boundary condition describing the clamped left side of the beam
u_D = np.array([0, 0, 0], dtype=ScalarType)
boundary_facets = mesh.locate_entities_boundary(
    domain, domain.topology.dim - 1, lambda x: np.isclose(x[0], 0)
)
bc = fem.dirichletbc(
    u_D, fem.locate_dofs_topological(V, domain.topology.dim - 1, boundary_facets), V
)

In [ ]:
problem = fem.petsc.NewtonSolverNonlinearProblem(F, u, bcs=[bc], graph=graph_)
solver = nls.petsc.NewtonSolver(MPI.COMM_WORLD, problem, graph=graph_)
solver.solve(u, graph=graph_)

In [ ]:
J_form = ufl.inner(u, u) * ufl.dx
J = fem.assemble_scalar(fem.form(J_form, graph=graph_), graph=graph_)

In [ ]:
dJdlambda = graph_.backprop(id(J), id(lambda_))
dJdmu = graph_.backprop(id(J), id(mu))

if MPI.COMM_WORLD.rank == 0:
    print("\n" + "=" * 60)
    print("Gradient Results:")
    print("=" * 60)
    print(f"J = {J}")
    print(f"dJ/dλ = {dJdlambda}")
    print(f"dJ/dμ = {dJdmu}")
    print("=" * 60)